<a href="https://colab.research.google.com/github/Kenohsi/ConnectFour/blob/master/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# ============================================
# ✅ FH CAMPUS WIEN QA CHATBOT – STABILE VERSION (Colab-kompatibel)
# Kein accelerate / kein Trainer – reines PyTorch-Training
# ============================================

# 1️ Umgebung aufräumen und korrekte Pakete installieren
!pip uninstall -y transformers accelerate
!pip install -q --no-cache-dir "transformers==4.36.2" "datasets==2.19.1" "torch" "sentencepiece" "gradio" "scikit-learn"

# -------------------------------------------------------
# 2️ Imports
# -------------------------------------------------------
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
import re
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import torch.nn.functional as F

# -------------------------------------------------------
# 3️ Daten laden (train.txt muss im /content liegen)
# -------------------------------------------------------
file_path = "/content/train.txt"
with open(file_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

# Extrahiere alle Frage-Antwort-Paare
pattern = r"### Q:\s*(.*?)\nA:\s*(.*?)(?=\n### Q:|$)"
pairs = re.findall(pattern, raw_text, re.DOTALL)
print(f"Loaded {len(pairs)} Q&A pairs.")

# -------------------------------------------------------
# 4️ Datensatz erstellen
# -------------------------------------------------------
data_dict = {"question": [], "context": [], "answers": []}
for q, a in pairs:
    q = q.strip()
    a = a.strip()
    data_dict["question"].append(q)
    data_dict["context"].append(a)
    data_dict["answers"].append({"text": [a], "answer_start": [0]})

dataset = Dataset.from_dict(data_dict)

# -------------------------------------------------------
# 5️ Modell & Tokenizer laden
# -------------------------------------------------------
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

# -------------------------------------------------------
# 6️ Vorverarbeitung (berechnet Start-/End-Tokens für Antwort)
# -------------------------------------------------------
def preprocess(example):
    tokenized = tokenizer(
        example["question"],
        example["context"],
        truncation=True,
        max_length=256,
        padding="max_length",
        return_offsets_mapping=True,
    )

    offsets = tokenized.pop("offset_mapping")
    answer = example["answers"]
    start_char = answer["answer_start"][0]
    end_char = start_char + len(answer["text"][0])
    sequence_ids = tokenized.sequence_ids()

    token_start_index = 0
    while sequence_ids[token_start_index] != 1:
        token_start_index += 1
    token_end_index = len(tokenized["input_ids"]) - 1
    while sequence_ids[token_end_index] != 1:
        token_end_index -= 1

    if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
        tokenized["start_positions"] = 0
        tokenized["end_positions"] = 0
    else:
        while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
            token_start_index += 1
        tokenized["start_positions"] = token_start_index - 1

        while token_end_index >= 0 and offsets[token_end_index][1] >= end_char:
            token_end_index -= 1
        tokenized["end_positions"] = token_end_index + 1

    return tokenized

tokenized_dataset = dataset.map(preprocess)

# -------------------------------------------------------
# 7️ Manuelles Fine-Tuning (statt Trainer)
# -------------------------------------------------------
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

def collate_fn(batch):
    # Konvertiere Listen zu Tensoren (alle gleich lang dank padding="max_length")
    input_ids = torch.tensor([b["input_ids"] for b in batch], dtype=torch.long)
    attention_mask = torch.tensor([b["attention_mask"] for b in batch], dtype=torch.long)
    start_positions = torch.tensor([b["start_positions"] for b in batch], dtype=torch.long)
    end_positions = torch.tensor([b["end_positions"] for b in batch], dtype=torch.long)
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "start_positions": start_positions,
        "end_positions": end_positions,
    }

train_loader = DataLoader(tokenized_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.train()

for epoch in range(2):  # 2 Trainingsdurchläufe
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    for batch in loop:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        start_positions = batch["start_positions"].to(device)
        end_positions = batch["end_positions"].to(device)

        optimizer.zero_grad()
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            start_positions=start_positions,
            end_positions=end_positions
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        loop.set_postfix(loss=loss.item())


# -------------------------------------------------------
# 8️ Modell speichern
# -------------------------------------------------------
model.save_pretrained("/content/fh-campus-qa-model")
tokenizer.save_pretrained("/content/fh-campus-qa-model")
print(" Model fine-tuned successfully and saved!")

# -------------------------------------------------------
# 9️ Test-Inferenz (funktioniert ohne pipeline)
# -------------------------------------------------------
def answer_qa(question: str, context: str, max_length: int = 256) -> str:
    # Tokenisierung inkl. Offsets
    enc = tokenizer(
        question,
        context,
        return_offsets_mapping=True,
        return_tensors="pt",
        truncation=True,
        max_length=max_length,
    )
    offsets = enc.pop("offset_mapping")[0]

    # sicherstellen, dass Modell & Input auf gleichem Gerät liegen
    device = "cuda" if torch.cuda.is_available() else "cpu"
    enc = {k: v.to(device) for k, v in enc.items()}
    model.to(device)

    # Vorwärtsdurchlauf ohne Gradientenberechnung
    with torch.no_grad():
        out = model(**enc)
        start_idx = torch.argmax(out.start_logits, dim=-1).item()
        end_idx = torch.argmax(out.end_logits, dim=-1).item()
        if end_idx < start_idx:
            start_idx, end_idx = end_idx, start_idx

    # Token-Offsets -> Text-Span
    start_char = int(offsets[start_idx][0])
    end_char = int(offsets[end_idx][1])
    start_char = max(0, min(start_char, len(context)))
    end_char = max(start_char, min(end_char, len(context)))

    return context[start_char:end_char].strip()

question = "Wo befindet sich die FH Campus Wien?"
context = "Die FH Campus Wien hat ihren Hauptstandort in der Favoritenstraße 232, 1100 Wien."
print("\n--- TEST ---")
print("Frage:", question)
print("Antwort:", answer_qa(question, context))

# -------------------------------------------------------
# 10 Gradio Chat-Interface mit TF-IDF-Retriever
# -------------------------------------------------------
import gradio as gr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

MODEL_PATH = "/content/fh-campus-qa-model"
TOKENIZER = AutoTokenizer.from_pretrained(MODEL_PATH)
MODEL = AutoModelForQuestionAnswering.from_pretrained(MODEL_PATH).eval()

with open("/content/train.txt", "r", encoding="utf-8") as f:
    raw = f.read()
pairs = re.findall(r"### Q:\s*(.*?)\nA:\s*(.*?)(?=\n### Q:|$)", raw, re.DOTALL)
QUESTIONS = [q.strip() for q,_ in pairs]
CONTEXTS  = [a.strip() for _,a in pairs]
CORPUS = [q + " " + c for q,c in zip(QUESTIONS, CONTEXTS)]
VECT = TfidfVectorizer(min_df=1, ngram_range=(1,2))
X = VECT.fit_transform(CORPUS)

def retrieve(question, k=1):
    sims = cosine_similarity(VECT.transform([question]), X)[0]
    idx = sims.argsort()[::-1][:k]
    return [(CONTEXTS[i], QUESTIONS[i], float(sims[i])) for i in idx]

def qa_answer(question, context, max_length=256):
    enc = TOKENIZER(question, context, return_offsets_mapping=True,
                    return_tensors="pt", truncation=True, max_length=max_length)
    offsets = enc.pop("offset_mapping")[0]
    with torch.no_grad():
        out = MODEL(**{k:v for k,v in enc.items()})
        s = torch.argmax(out.start_logits, dim=-1).item()
        e = torch.argmax(out.end_logits, dim=-1).item()
        if e < s: s, e = e, s
    start_char = int(offsets[s][0]); end_char = int(offsets[e][1])
    start_char = max(0, min(start_char, len(context)))
    end_char   = max(start_char, min(end_char, len(context)))
    return context[start_char:end_char].strip()

def respond(message, history):
    text = message.lower().strip()

    if text in ["hi", "hallo", "hey", "servus", "moin", "hello"]:
        return "Hallo! Ich bin der FH‑Campus‑Wien‑Bot 🙂 Frag mich gern etwas zur FH Campus Wien."

    hits = retrieve(message, k=1)
    ctx, matched_q, score = hits[0]

    # wenn die Ähnlichkeit sehr niedrig ist -> lieber ehrlich sein
    if score < 0.2:
        return "Zu dieser Frage habe ich in meinen FH‑Unterlagen leider nichts Passendes gefunden."

    ans = qa_answer(message, ctx)
    return ans


demo = gr.ChatInterface(
    fn=respond,
    title="FH Campus Wien QA",
    description="Stell eine Frage – das Modell sucht den besten Kontext aus train.txt und extrahiert die Antwort."
)

demo.launch(share=True)

Found existing installation: transformers 4.36.2
Uninstalling transformers-4.36.2:
  Successfully uninstalled transformers-4.36.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 21.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.18.0 requires accelerate>=0.21.0, which is not installed.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.2 which is incompatible.
Loaded 79 Q&A pairs.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/79 [00:00<?, ? examples/s]

Epoch 1:   0%|          | 0/40 [00:00<?, ?it/s]

Epoch 2:   0%|          | 0/40 [00:00<?, ?it/s]

 Model fine-tuned successfully and saved!

--- TEST ---
Frage: Wo befindet sich die FH Campus Wien?
Antwort: Die FH Campus Wien hat ihren Hauptstandort in der Favoritenstraße 232, 1100 Wien.


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c0d0619276ed58b1c0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [5]:
# @title
with open("train.txt", "w", encoding="utf-8") as f:
    f.write("""### Q: Was ist die FH Campus Wien?
A: Die FH Campus Wien ist die größte Fachhochschule in Österreich mit ca. 8.000 Studierenden und bietet etwa 70 Studienprogramme in sieben Departments an. [1](https://www.uni.at/uni/hochschule-campus-wien/)[2](https://www.hcw.ac.at/)

### Q: Welche Departments gibt es an der FH Campus Wien?
A: Sieben Departments: Angewandte Pflegewissenschaft, Applied Life Sciences, Bauen und Gestalten, Gesundheitswissenschaften, Sozialwissenschaften, Technik sowie Verwaltung, Wirtschaft, Sicherheit, Politik. [1](https://www.uni.at/uni/hochschule-campus-wien/)[2](https://www.hcw.ac.at/)

### Q: Wie viele Studienprogramme bietet die FH Campus Wien an?
A: Es gibt knapp 70 Studienprogramme – Bachelor, Master, berufsbegleitend, dual – an Vollzeit- und Teilzeitstandorten. [1](https://www.uni.at/uni/hochschule-campus-wien/)[2](https://www.hcw.ac.at/)

### Q: Wo befindet sich die FH Campus Wien?
A: Die FH Campus Wien hat ihren Hauptstandort in der Favoritenstraße 232, 1100 Wien. [1](https://www.uni.at/uni/hochschule-campus-wien/)

### Q: Wie läuft das Aufnahmeverfahren an der FH Campus Wien ab?
A: Der Prozess umfasst meist: Online-Registrierung, schriftlichen Aufnahmetest (logisch, verbal, numerisch), studiengangsabhängig praktischen Test und/oder Bewerbungsgespräch bzw. Gruppeninterview. [3](https://www.aufnahmepruefung.at/fh-campus-wien/)[4](https://www.hcw.ac.at/studium-weiterbildung/bewerbung-und-aufnahme)

### Q: Welche Zugangsvoraussetzungen sind nötig?
A: Matura (Reifeprüfung) oder äquivalente Prüfung (z. B. Berufsreifeprüfung oder Studienberechtigungsprüfung). Außerdem ist Deutsch- bzw. Englisch-Niveau oft erforderlich; bei ausländischen Abschlüssen müssen beglaubigte Übersetzungen eingereicht werden. [4](https://www.hcw.ac.at/studium-weiterbildung/bewerbung-und-aufnahme)

### Q: Was passiert, wenn ich keine Matura habe?
A: Du kannst die Berufsreifeprüfung oder eine Studienberechtigungsprüfung ablegen. Alternativ kann auch berufliche Qualifikation plus Zusatzprüfungen anerkannt werden. [4](https://www.hcw.ac.at/studium-weiterbildung/bewerbung-und-aufnahme)

### Q: Gibt es unterschiedliche Bewerbungsfristen?
A: Ja, Fristen hängen vom Studiengang ab. Anmeldung erfolgt online, und spezifische Termine für Tests und Gespräche sind zuerst-kommen, zuerst-gewählt. [4](https://www.hcw.ac.at/studium-weiterbildung/bewerbung-und-aufnahme)[3](https://www.aufnahmepruefung.at/fh-campus-wien/)

### Q: Welche Studiengänge im Bereich Technik werden angeboten?
A: Beispiele: Bachelor Computer Science and Digital Communications (Vollzeit + berufsbegleitend), High Tech Manufacturing, Clinical Engineering, Angewandte Elektronik & Technische Informatik. [5](https://www.hcw.ac.at/studium-weiterbildung/studienangebot)

### Q: Welche Bildungsarten werden angeboten?
A: Bachelor, Master, Weiterbildungsstudium und akademische Hochschullehrgänge. Lehrformen: Vollzeit, berufsbegleitend, dual; Unterrichtssprachen sind Deutsch und Englisch. [5](https://www.hcw.ac.at/studium-weiterbildung/studienangebot)

### Q: Welche Studienrichtungen im Bereich Gesundheit gibt es?
A: Beispiele: Pflegewissenschaft, Gesundheits- und Krankenpflege, Applied Life Sciences, Klinische Forschung.

### Q: Gibt es duale Studienmöglichkeiten?
A: Ja, einige Studiengänge können dual absolviert werden, z. B. Technik oder Wirtschaft, in Zusammenarbeit mit Unternehmen.

### Q: Wie melde ich mich zum Aufnahmetest an?
A: Die Anmeldung erfolgt online über das Bewerbungsportal der FH Campus Wien.

### Q: Wie lange dauert der Aufnahmetest?
A: Der schriftliche Aufnahmetest dauert in der Regel 60–90 Minuten, je nach Studiengang.

### Q: Kann ich mich für mehrere Studiengänge gleichzeitig bewerben?
A: Ja, eine Mehrfachbewerbung ist möglich, solange die Bewerbungsfristen eingehalten werden.

### Q: Welche Abschlussarten gibt es an der FH Campus Wien?
A: Bachelor, Master, akademische Hochschullehrgänge und Weiterbildungsstudien.

### Q: Gibt es internationale Studiengänge?
A: Ja, einige Programme werden in Kooperation mit internationalen Partnern in Englisch angeboten.

### Q: Wie kann ich ein Praktikum absolvieren?
A: Praktika werden entweder curricular vorgeschrieben oder freiwillig in Zusammenarbeit mit Unternehmen angeboten.

### Q: Welche IT-Ressourcen stehen Studierenden zur Verfügung?
A: Studierende haben Zugang zu Computerräumen, Softwarelizenzen, WLAN und Lernplattformen.

### Q: Gibt es studentische Organisationen oder Clubs?
A: Ja, Studierende können sich in Fachschaften, Sportvereinen oder Projektgruppen engagieren.

### Q: Welche Prüfungen muss ich ablegen?
A: Prüfungen variieren je nach Studiengang und können schriftlich, mündlich oder praxisbasiert sein.

### Q: Gibt es Online-Kurse oder E-Learning-Angebote?
A: Ja, die FH Campus Wien bietet ausgewählte Lehrveranstaltungen auch online an.

### Q: Wie beantrage ich eine Studienbeihilfe?
A: Studienbeihilfen werden über das Bundesministerium für Bildung und über das Studierendenportal beantragt.

### Q: Kann ich mein Studium unterbrechen?
A: Ja, ein Urlaubssemester kann unter bestimmten Bedingungen beantragt werden.

### Q: Wie viele Studienplätze gibt es pro Jahrgang?
A: Die Anzahl der Studienplätze hängt vom Studiengang ab, meist zwischen 20 und 200 Plätzen pro Jahrgang.

### Q: Gibt es Zugangsvoraussetzungen für Masterstudiengänge?
A: Ja, in der Regel wird ein Bachelorabschluss oder ein gleichwertiger Abschluss vorausgesetzt.

### Q: Wie lange dauert ein Masterstudium?
A: Ein Masterstudium dauert in der Regel 2 Jahre (4 Semester) in Vollzeit.

### Q: Gibt es Praxisprojekte im Studium?
A: Ja, viele Studiengänge beinhalten Praxisprojekte, Laborübungen oder Fallstudien.

### Q: Kann ich im Ausland studieren?
A: Ja, die FH Campus Wien kooperiert mit internationalen Partnerhochschulen für Austauschprogramme.

### Q: Welche Sprachen muss ich für die Zulassung nachweisen?
A: Deutschkenntnisse sind in der Regel Pflicht, Englischkenntnisse je nach Studiengang.

### Q: Gibt es Studienberatung?
A: Ja, die FH Campus Wien bietet individuelle Beratung für Studieninteressierte an.

### Q: Wie hoch sind die Studiengebühren?
A: Die Studiengebühren variieren je nach Studiengang; für österreichische Studierende meist moderat.

### Q: Gibt es Stipendien für internationale Studierende?
A: Ja, internationale Studierende können auf spezifische Förderungen und Stipendien zugreifen.

### Q: Wie melde ich mich für Lehrveranstaltungen an?
A: Die Anmeldung erfolgt online über die Lernplattform der FH Campus Wien.

### Q: Gibt es einen Campusplan oder Gebäudeübersicht?
A: Ja, die FH Campus Wien stellt einen detaillierten Campusplan online zur Verfügung.

### Q: Was ist die FH Campus Wien?
A: Die FH Campus Wien ist die größte Fachhochschule Österreichs mit rund 8.000 Studierenden.

### Q: Wo befindet sich die FH Campus Wien?
A: In Wien-Favoriten, nahe der U1-Station Altes Landgut.

### Q: Welche Studiengänge gibt es?
A: Zum Beispiel Technik, Gesundheit, Soziales und Wirtschaft.

### Q: Wie läuft das Aufnahmeverfahren ab?
A: Meist ein schriftlicher Test und ein Bewerbungsgespräch.

### Q: Gibt es ein Masterstudium?
A: Ja, in mehreren Bereichen wie Health Assisting Engineering oder Sozialmanagement.

### Q: Welche U-Bahn-Station liegt am nächsten zur FH Campus Wien?
A: Die nächstgelegene Station ist „Altes Landgut“ auf der U1, nur wenige Gehminuten vom Haupteingang entfernt.

### Q: Gibt es Parkmöglichkeiten an der FH Campus Wien?
A: Ja, es gibt eine Tiefgarage sowie begrenzte Außenparkplätze, die jedoch kostenpflichtig sind.

### Q: Gibt es Mensen oder Cafés am Campus?
A: Ja, der Campus hat eine große Mensa, mehrere Cafés und Snackautomaten für Studierende und Mitarbeiter.

### Q: Wie melde ich mich für ein Auslandssemester an?
A: Die Anmeldung erfolgt über das International Office der FH Campus Wien. Dort erhältst du Infos zu Partnerunis, Bewerbungsfristen und Förderprogrammen.

### Q: Welche Partneruniversitäten hat die FH Campus Wien?
A: Die FH kooperiert mit zahlreichen Hochschulen in Europa, Asien und Nordamerika – genaue Listen stehen im International-Office‑Bereich der FH-Website.

### Q: Gibt es ein Tutorenprogramm für Erstsemestrige?
A: Ja, viele Studiengänge bieten Mentor*innen oder Tutor*innen, die Erstsemestrige im ersten Semester begleiten und unterstützen.

### Q: Wie funktioniert die Prüfungsanmeldung?
A: Die Prüfungsanmeldung erfolgt über die Online‑Plattform der FH Campus Wien, meist einige Wochen vor der Prüfung.

### Q: Wann beginnt das Studienjahr an der FH Campus Wien?
A: Das Studienjahr startet in der Regel im September mit der ersten Semesterwoche.

### Q: Gibt es ein Sommersemester?
A: Ja, der zweite Studienabschnitt beginnt im Februar und endet im Juni.

### Q: Welche Sprachkenntnisse brauche ich für englische Studiengänge?
A: Meist wird ein Englisch‑Level von B2 oder höher benötigt, oft durch Zertifikate wie IELTS oder TOEFL nachweisbar.

### Q: Gibt es Barrierefreiheit am Campus?
A: Ja, der Campus ist vollständig barrierefrei, inklusive Aufzügen, Rampe und barrierefreien Sanitäranlagen.

### Q: Wie funktioniert die Bibliotheksausleihe?
A: Studierende können Bücher online reservieren und vor Ort abholen. Die Ausleihdauer variiert je nach Medium.

### Q: Kann ich Drucker und Kopierer verwenden?
A: Ja, die FH bietet Druck‑ und Kopierstationen, die über ein Guthabensystem funktionieren.

### Q: Gibt es WLAN am gesamten Campus?
A: Ja, es gibt flächendeckendes WLAN für Studierende, erreichbar über den FH‑Account.

### Q: Wie trete ich der Fachschaft bei?
A: Jede Fachschaft sucht regelmäßig neue Mitglieder; Interessierte können sich direkt über die Fachschaftsseite melden.

### Q: Gibt es Sportangebote für Studierende?
A: Ja, es werden Sportkurse, Fußballturniere und andere Aktivitäten organisiert, meist über den FH‑Sportverein.

### Q: Welche Softwares stehen Studierenden zur Verfügung?
A: Je nach Studium erhalten Studierende Zugriff auf Software wie MATLAB, SolidWorks, SPSS, Office 365 und weitere Fachtools.

### Q: Wie finde ich meine Stundenpläne?
A: Stundenpläne sind im jeweiligen Studiengangsportal online einsehbar und werden vor Semesterbeginn veröffentlicht.

### Q: Gibt es Labore am Campus?
A: Ja, zahlreiche Labore stehen zur Verfügung – z. B. Elektrotechnik, Biomedizin, Bioanalytik, Bauingenieurwesen und Pflegewissenschaft.

### Q: Wie läuft die Zulassung nach der Bewerbung ab?
A: Nach bestandenen Tests und Interviews erhält man eine offizielle Zusage per E‑Mail sowie einen Studienvertrag.

### Q: Welche Unterlagen brauche ich für die Bewerbung?
A: Meist Matura‑Zeugnis, Lebenslauf, Ausweis, Motivationsschreiben und eventuelle Zertifikate.

### Q: Kann ich mein Studium wechseln?
A: Ein Studienwechsel ist möglich, aber abhängig von Fristen und Anerkennung von Vorleistungen.

### Q: Welche Masterprogramme im Bereich Wirtschaft gibt es?
A: Unter anderem Public Management, Risiko‑ und Sicherheitsmanagement oder Digital Business Innovation.

### Q: Welche Rolle spielt Praxis im Studium?
A: Fast alle Studiengänge beinhalten verpflichtende Praxisphasen, Praktika oder Projekte.

### Q: Wie hoch ist die Anwesenheitspflicht?
A: Viele Lehrveranstaltungen an Fachhochschulen haben Präsenzpflicht, je nach Studiengang zwischen 50–100 %.

### Q: Wie funktioniert die Kommunikation mit Lehrenden?
A: Über E‑Mail, die Lernplattform und persönliche Termine. Viele Lehrende haben fixe Sprechstunden.

### Q: Gibt es Möglichkeit für studentische Jobs?
A: Ja, am schwarzen Brett der FH und online werden regelmäßig Jobs und Praktika ausgeschrieben.

### Q: Wie kann ich meine Noten einsehen?
A: Jede Note wird im Campus‑Management‑System online eingetragen und ist dort abrufbar.

### Q: Was passiert, wenn ich eine Prüfung nicht bestehe?
A: Es gibt mehrere Wiederholungsmöglichkeiten, die jeweils im Studienhandbuch geregelt sind.

### Q: Gibt es ein Career Center?
A: Ja, das Career Center bietet Beratung, Workshops, Bewerbungstipps und Kontakte zu Unternehmen.

### Q: Gibt es Forschung an der FH Campus Wien?
A: Ja, die FH betreibt angewandte Forschung in Bereichen wie Technik, Gesundheit, Life Sciences und Soziales.

### Q: Welche Vorteile bietet ein FH‑Studium?
A: Viel Praxisbezug, fixe Studienplätze, klare Struktur und enge Zusammenarbeit mit Unternehmen.

### Q: Wie groß sind typische Gruppen?
A: 20 bis 40 Studierende pro Jahrgang, abhängig vom Studiengang.

### Q: Gibt es Lernräume oder Gruppenarbeitsplätze?
A: Ja, es gibt mehrere Lernzonen, Gruppenräume und ruhige Arbeitsplätze im gesamten Campus.

### Q: Wie funktioniert die Abwesenheitsmeldung?
A: Abwesenheiten müssen je nach Studiengang im Portal eingetragen oder bei Lehrenden gemeldet werden.

### Q: Gibt es Weiterbildungsmöglichkeiten nach dem Bachelor?
A: Ja, zahlreiche Masterstudien und berufsbegleitende Lehrgänge stehen zur Auswahl.

### Q: Kann ich als internationaler Student an der FH Campus Wien studieren?
A: Ja, internationale Bewerber sind willkommen, benötigen jedoch Anerkennungen ihrer Zeugnisse und Sprachnachweise.

### Q: Wie läuft der Starttag für Erstsemestrige ab?
A: Am Anfang gibt es oft eine Begrüßungsveranstaltung, Einführung in Systeme und Kennenlernen des Campus.

### Q: Gibt es digitale Lehrveranstaltungen?
A: Ja, einige Kurse werden hybrid oder vollständig online angeboten, je nach Studienrichtung.



""")
